# Knihovna CtiOSGdal

Nejdrive naimportujeme knihovny:

In [1]:
import os
import sys
import sqlite3
import tempfile
from shutil import copyfile

# Moduly jsou v jinem adresari, je tedy nutne tento adresar pridat do sys.path
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
   sys.path.append(module_path)

from services.ctiOS.gdal import CtiOSGdal

Pote nadefinujeme parametry nutne pro inicializaci knihovny:

In [2]:
# pristupove testovaci udaje k sluzbam WSDP
username = "WSTEST"
password = "WSHESLO"
# rozbaleny VFK soubor ve forme sqlite databaze
db_path = os.path.join(module_path, 'services', 'ctiOS', 'data', 'input', 'Export_1-4.db')
print(db_path)

/home/linwe/pywsdp/services/ctiOS/data/input/Export_1-4.db


### Zpracovani cele tabulky OPSUB

Nejprve nas bude zajimat zpracovani cele tabulky OPSUB - tedy doplneni udaju ke vsem radkum v teto tabulce. V databazi jsou tyto udaje zatim nevyplnene:

In [3]:
con = sqlite3.connect(db_path)
cur = con.cursor()

for row in cur.execute("SELECT * FROM OPSUB"):
    print(row)
    
con.close()

('64R47PeqD9SONivn8apY/lJWp7PMqXzY9utJl+M+BfUUirKmMZliVuxYKEtLpIva6glOyFHdvIofu9Hwu2HKl1m5M6qSyaHfCuQu6GdIy0E=', None, None, None, None, None, None, None, None, None, 'OPO', 3, 1, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1)
('cWAf13LtjvC9SIY05qIFaek9K/RpuqJao0AMXiHsDYFjM2eS251ouFycwesABoDs2wWu+kFTmYqxhJyeR7e+WrdGQBbrYHk4M21Ai3xMSF4=', None, None, None, None, None, None, None, None, None, 'OFO', 2, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 2)
('psGq9l14Woi29AYN5bHwdxU5WmX1vyRzVSJuucNo5e2D8193gu/to3VObZbd3vc0L+fDzX+X5nL42Wu8iePZgJhokFKOM8uNqezVZiQQ6P0=', None, None, None, None, None, None, None, None, None, 'OFO', 2, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 3)
('r/e02ptfcJ47n0

Abychom neztratili puvodni nevyplnena data, udelame si bokem kopii databaze, kterou pak budeme editovat:

In [4]:
temp_db = tempfile.mkstemp()[1]
copyfile(db_path, temp_db)
print(temp_db)

/tmp/tmpyi_1mbu7


Zavolame knihovnu a ziskame vsechny identifikatory z databaze ve forme seznamu:


In [5]:
ctios = CtiOSGdal(username=username,
                  password=password,
                  db_path=temp_db)
posidents = ctios.get_posidents_from_db()
print(posidents)
len(posidents)

['<v2:pOSIdent>aOWF/BYRV4K+0xg/HdeP3T3Tm/aSXL8Z3ShJxg4zYChQJRi3EoVsQ1xr+eatOjsz8o55pTpCs81iSwIC5oNvPFZhZ5upIuj6y1Ntb5sYT1M=</v2:pOSIdent>', '<v2:pOSIdent>bALPx2GbQF+oclIHg6WmRdXh3VGeifIWFvCKDsbvenLQNSYNrm6stXbIt7ywtIFTKcJ8K0vSAfKeokpiRi0l3Aex77vwTqobXY/c2lb1xw0=</v2:pOSIdent>', '<v2:pOSIdent>MvCbj6Q05AQW6847EAgIYrqDl8FsW8iuqz8WE/HYFlpxpTCi+ZWTWE8dAPeAWfTMF1wjj1diDvFHVemk8/36dxQsW43Uru7qNvUgjA0dTLp6FJD3zaBDiCU/SbyoKdsU</v2:pOSIdent>', '<v2:pOSIdent>yqpT3s/QygUWqCsg/azKypTOTOuJdbjB/16kQv0m5uugaHFJh8Mtp2i7sCdt71GRZixdYAzCVqEa4mNgvEjDECGf0Fby1B2nv1NdlyFQPrg=</v2:pOSIdent>', '<v2:pOSIdent>5XEbVUHVfI6OxlRPfJGQH6zTwljfLIEzxpL2+mYKOAkIlVMDACBfwbGRkxThLw94hZsh7wZCDRSKiMSbaMMEXuSHd/RO5NZXuOsJqhYGlKI=</v2:pOSIdent>', '<v2:pOSIdent>EGgfRTDVb68syXhEmpfl6VpmxItEm1IwCtdk82EaSNgBGKDd5gEbCPEBw6wrbaGWim1nDiiVsb1iarVPI/2OAVvMYg3BxyQUKFuMpareq+c=</v2:pOSIdent>', '<v2:pOSIdent>mSH6sS5yv6vA2jEOtyHeXbjjmSLmJ6SkHkWy/unkzDUZk7xIPT4ioZT0NeRHDgnO9rulMBaBITs2aW7PRZ1GTMRq6GAz0Ie+yIhff+XnzY8=</v2:pOSIdent>', '<v2:p

108

Nad seznamem zavolame metodu .process, ktera k zadanym identifikatorum aktualizuje v tabulce osobni udaje:

In [6]:
ctios.process(posidents)

ctiOS        - INFO     - POSIDENT aOWF/BYRV4K+0xg/HdeP3T3Tm/aSXL8Z3ShJxg4zYChQJRi3EoVsQ1xr+eatOjsz8o55pTpCs81iSwIC5oNvPFZhZ5upIuj6y1Ntb5sYT1M= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT bALPx2GbQF+oclIHg6WmRdXh3VGeifIWFvCKDsbvenLQNSYNrm6stXbIt7ywtIFTKcJ8K0vSAfKeokpiRi0l3Aex77vwTqobXY/c2lb1xw0= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT MvCbj6Q05AQW6847EAgIYrqDl8FsW8iuqz8WE/HYFlpxpTCi+ZWTWE8dAPeAWfTMF1wjj1diDvFHVemk8/36dxQsW43Uru7qNvUgjA0dTLp6FJD3zaBDiCU/SbyoKdsU USPESNE STAZEN
ctiOS        - INFO     - POSIDENT yqpT3s/QygUWqCsg/azKypTOTOuJdbjB/16kQv0m5uugaHFJh8Mtp2i7sCdt71GRZixdYAzCVqEa4mNgvEjDECGf0Fby1B2nv1NdlyFQPrg= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT 5XEbVUHVfI6OxlRPfJGQH6zTwljfLIEzxpL2+mYKOAkIlVMDACBfwbGRkxThLw94hZsh7wZCDRSKiMSbaMMEXuSHd/RO5NZXuOsJqhYGlKI= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT EGgfRTDVb68syXhEmpfl6VpmxItEm1IwCtdk82EaSNgBGKDd5gEbCPEBw6wrbaGWim1nDiiVsb1iarVPI/2OAVvMYg3BxyQUKFuMpareq+c= USPESNE STAZEN
ctiOS        - INFO     - 

ctiOS        - INFO     - POSIDENT m+o3Qoxrit4ZwyJIPjx3X32uMv1tXAioo/hUUaq6JmMHGXsvc9zncnGPExVsV8dslyzXyBs7yEV6mKlHKK2k7OW4tsyI1O91ftE7Rl4B6B4= USPESNE STAZEN
ctiOS        - INFO     - Radky v databazi u POSIdentu 5wQRil9Nd5KIrn5KWTf8+ucfKEsayz4LR/ZrAucExc8z+4hWyMyh+ABIewfIBvgQMxxF8/2HqFMn0OE/IphQisnADW27BjQ8Cn2Ui2i+xLM= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu 7TsaZA7GFeEVxMscggvr2LkRAZwqfDywZ9HtkoX92fdNI/VFseD7SjfY8rJ7IlS7o2MG7D4mG56FZUTFOUWW7a5GeMmtDAasGLtodOgvIOA= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu 00ep7V5fFBZoFMeOfcAOPMRtu0G9svLcsDyC/fYkJk3EbFIqbw2kpRaC1jjS1V8bjacEwrp/SdxXZXdtwDzFj3vKzXvHF7k7HiDpVtv6fPAUIS/cKLcyZ7viQd7pE5FE aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu 4/09lLuaNar2b8ZrhcyqhU1dN1h6cdfYNCg2/RReim82UXm3H5whG2MlvoKzXFltwKfA5MonRqd32zCvZgGfvCUIbqRScDVWKy5fctYb6N0= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu PtV2GbyZOZzpReCKf7eFnHRmrU7K4TTcUse25gDBLjKDnzwoc8Sz5t

ctiOS        - INFO     - Radky v databazi u POSIdentu iXuo5APMiSDKHjvq6Zwp4znpylRk3VICHbMd0CZEnHv0Yfd9IxzLRCvYD0GcdPBgCsVQP5C/sjWn2YmF1qe/BxyobmNhG4SPsa0pKedo4PE= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu AGLNrRX8gxMGgFxc57odWW/yGmn7NeHtyEPqURGIoazP16SegE6xwFHfWC/gQY+WgTtLKjG3cPMzRltvADCgso5adynZFpiklJffFOc7sSo= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu xairWnG248pu7itR19AIT0hdzsYi1LAHuAt8fIswNgUvi97XTCdWljEuXGf2xQ4c5hhgGnY0ZVp2yOEEGwJ70iljjCDlCM46qs/+W/UHZSQ= aktualizovany
ctiOS        - INFO     - POSIDENT BBgBQ2wok0L43hEZUEeiehwwgrnYkvSWpgVkTEuS44FGAKrsjBz/UVx+LWdeKO2khvGuk0dwf2GE9ziJR5j2lVd4ioIVbPKS1oaCurIfCpA= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT Ku76SlYfgFivONmh8bdyl02qHhLEuGPdDbCow70bilmRc26J2noXqBoufBPT4kfi2B5Nm5H9w1NFHR+pNQuGoQfdXwnXwz8VCNF2QsJnBe8ypwQghGm+6xd1Xp/20VgA USPESNE STAZEN
ctiOS        - INFO     - POSIDENT l0raZybOozQ95JWgTnaQpiplmmtdzAp2RiExSTwc+CmrRcDkW8jPHrWwYA33OP9DV3647Ea9f04G8Wyca2N6FJccnQQNF

ctiOS        - INFO     - POSIDENT NezMzNG/s2Rz33QBJ/J9xHsKkt3gPqkDcB/VqU6fmfWs/kc8bLBXqEfKbgE9/Co4Ah9Ft2xfVdjm9TZnDDLf0DSivtf34CqbrOgDPJv7Q/c= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT hMFmuv9xLKl7itElRb+ABKUfR4GPjNp8o8f7v5TImNsrDMbOSFs0n+msxztqnzvgTnj+Rm6QehwZNK64c6pIeSUBViUprt4npOhmD/7zcrk= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT neqUuNVqO4j95P+NofCPKL0MpZxkXTUrIQe/eQxf0fZUF4yLhcArvN3to7uRIMN7sVm18i3/pvS0bQbpHxmQT+iN8Gfl/dcR2gsSyVVLXGs= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT mm/dPuBRthrYVYeJFVs7KbJF8/0AcMBynbtvmHypm+QNWR7vTP4qB6Fjx93b2q7on9TIgT5u2hb9//QrjDl7NRJCaZYrCLqylJUlk4wyvV6WfgnUKGSoDqXg3KnMyIO9 USPESNE STAZEN
ctiOS        - INFO     - POSIDENT cc3RAb7ln/HyHThRavcVRYDuQQHgv9VV8nNlwdBNLUZ6oCPq/qyk+T1vPYEx0GCJEl+tQ7UbtUZMwecAoFcEAqknnyNkwfherStqUhxxEQ8= USPESNE STAZEN
ctiOS        - INFO     - Radky v databazi u POSIdentu 3aW3srZ5a3T/ChIj+sdMrJE0sk738z0ZHA0q6GgFh5H2Szl4VYLSRCfCHyEMA8cAKwbYizIHXcv/r5goxydoDSZCgYHe6mOZsomLL2Jwop4= aktualizovany
ctiOS  

ctiOS        - INFO     - Radky v databazi u POSIdentu psGq9l14Woi29AYN5bHwdxU5WmX1vyRzVSJuucNo5e2D8193gu/to3VObZbd3vc0L+fDzX+X5nL42Wu8iePZgJhokFKOM8uNqezVZiQQ6P0= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu Ym8cWfYJCf8Urrv3bW6nwRZlwSmAaA/8kS8GFhllL8eFIJFCj0FhuzDnHm4d/DQi+nop3gtaovf0HR4b7x9LbCQdjj8cfjGO0N/jFAM+vK4= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu FxrmEj8jkUJ9rfUt8DkkVOUbhQdxwKgENy+oxwXMp0yU3OttGGEKPmExw49PdHUzB2Fwto/UgZ+MAHpoXFEeOWwwuK+acbAp2CPWPzhTl9A= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu fwdsUaxtVKyjWdhcx8QMu3passZadTC0q5Sr0L0GOPWU1zzK73MDWdQ5MBficQF1IrTyXx7Rs9OwGPxW1syorUZI5+2oQKWwn68UnX0PwWE= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu Y+s9bH0/ld/NLE4dOgaHgoRcmRrSt+dsqPhNnh0oXeafk9E4WZlgsCnF1La1gw7EoZ3hLsizggDDnbV+DyNJinyL5ABL87B4d4Q1NIT8goQ= aktualizovany
ctiOS        - INFO     - Radky v databazi u POSIdentu g4EDGsOOSJKaTbLxdmmIcj8xUOiCckDSBeYZG2QP8Pyo2wimkedJNVI

Pri posilani pozadavku na sluzbu mohou byt na strane WSDP sluzby odhaleny chyby ve vstupnim souboru: neplatny identifikator, expirovany identifikator a opravneny subjekt neexistuje. Jak muzeme videt, na konci zpracovani se nam vypise jednoducha statistika zpracovavanych subjektu obsahujici i statistiku chyb. Tuto statistiku je treba po kazdem zpracovani zkontrolovat.

 Nyni uz se muzeme podivat do naplnene databaze:

Podivame se do naplnene databaze:

In [7]:
con = sqlite3.connect(temp_db)
cur = con.cursor()

for row in cur.execute("SELECT * FROM OPSUB"):
    print(row)
    
con.close()

('64R47PeqD9SONivn8apY/lJWp7PMqXzY9utJl+M+BfUUirKmMZliVuxYKEtLpIva6glOyFHdvIofu9Hwu2HKl1m5M6qSyaHfCuQu6GdIy0E=', 0, '2000-01-01T00:00:00', None, 3, 78189317010, None, None, None, None, 'OPO', 3, 1, 1, 'Česká republika', 'CESKA REPUBLIKA', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1, '627994601')
('cWAf13LtjvC9SIY05qIFaek9K/RpuqJao0AMXiHsDYFjM2eS251ouFycwesABoDs2wWu+kFTmYqxhJyeR7e+WrdGQBbrYHk4M21Ai3xMSF4=', 0, '2000-01-01T00:00:00', None, 3, 54816151010, None, None, None, None, 'OFO', 2, None, None, None, None, '1606112981', None, 'Lumír', 'LUMIR', 'Hoza', 'HOZA', None, 1660, None, None, 'Hovězí', 'Komárov', None, None, 11000, None, 1, None, None, None, None, 2, '420485306')
('psGq9l14Woi29AYN5bHwdxU5WmX1vyRzVSJuucNo5e2D8193gu/to3VObZbd3vc0L+fDzX+X5nL42Wu8iePZgJhokFKOM8uNqezVZiQQ6P0=', 0, '2000-01-01T00:00:00', None, 3, 35758491010, None, None, None, None, 'OFO', 2, None, None, None, None, '0305263299', 

Skvele! Databazi se podarilo aktualizovat. :-)

### Vyber pseudonymizovanych identifikatoru

Pokud nepotrebujeme zpracovavat celou tabulku, muzeme si vybrat jen nektere pseudonymizovane identifikatory.

Zase si udelame kopii databaze:

In [8]:
temp_db = tempfile.mkstemp()[1]
copyfile(db_path, temp_db)
print(temp_db)

/tmp/tmpborfr532


Zavolame si knihovnu a pri ziskavani identifikatoru z databaze pouzijeme parametr "sql". Zajima nas pouze 10 prvnich identifikatoru:

In [9]:
ctios = CtiOSGdal(username=username,
                  password=password,
                  db_path=temp_db)
sql = "SELECT * FROM OPSUB order by ID LIMIT 10"
posidents = ctios.get_posidents_from_db(sql) # pouzijeme parametr sql
print(posidents)

['<v2:pOSIdent>078yf28ANhKnYAoAfBZk6LwrtgWykcrWv2Inb0RySck69cjrSm5AFefIPtsNw/M2svL/JsumXXsklMRnHtccupBZ+KOedZqCraKOLujSdsc=</v2:pOSIdent>', '<v2:pOSIdent>3ZendyrB5RiGiS5emPZ8oWRhZZQOsLiOHI329ANsTWE3+RDPfNFmHeJzpKvspfBZpBX5RpwGNirjH3hlHeysozmb3+g52m7EfdnRYUR/iAA=</v2:pOSIdent>', '<v2:pOSIdent>+lQ0Ne+Jhd5K4uvds1/KT8qqJcoA/eJ5OloIoqvCQi04X93kbmO62f/i2fqJgk62odrpl2ye04ue7tlKR93hckH4u46kVMoHQWmYF/rBWMA=</v2:pOSIdent>', '<v2:pOSIdent>/ipU68hnjrq1J7ZDi4JBaAyBiKjxZzpGZSVIfv8yFMKSKy3PpT/lba/0c4YCdqvrexVCCAL0MxBbob3A75l2Tnkgw/HHXtm0+abSHVywLEM=</v2:pOSIdent>', '<v2:pOSIdent>1icyYJRr8366DjQYyqdj/TTYUgGOuUSp6ck6NDaYLdQ6lU3sbdOjY6CKYEN5T6XuJ8mBowWJicz/4l0lrv0Y+AZetgmFuW0OoM7yEd+5evQ=</v2:pOSIdent>', '<v2:pOSIdent>2fa9YZ/g+IACqr1Etk+QIUPK51iiTwCA/t/nTw+U9RUmjfYorPLrzpPd46Bmj3C3tfw3cyOy5bx0ogGCOZ01eUctqHZQzXY903JlPvzPzlw=</v2:pOSIdent>', '<v2:pOSIdent>00ep7V5fFBZoFMeOfcAOPMRtu0G9svLcsDyC/fYkJk3EbFIqbw2kpRaC1jjS1V8bjacEwrp/SdxXZXdtwDzFj3vKzXvHF7k7HiDpVtv6fPAUIS/cKLcyZ7viQd7pE5FE</v2:pOSIdent>', '<v2:p

Nad seznamem zavolame metodu .process, ktera k zadanym identifikatorum aktualizuje v tabulce osobni udaje:

In [10]:
ctios.process(posidents)

ctiOS        - INFO     - POSIDENT 078yf28ANhKnYAoAfBZk6LwrtgWykcrWv2Inb0RySck69cjrSm5AFefIPtsNw/M2svL/JsumXXsklMRnHtccupBZ+KOedZqCraKOLujSdsc= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT 3ZendyrB5RiGiS5emPZ8oWRhZZQOsLiOHI329ANsTWE3+RDPfNFmHeJzpKvspfBZpBX5RpwGNirjH3hlHeysozmb3+g52m7EfdnRYUR/iAA= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT +lQ0Ne+Jhd5K4uvds1/KT8qqJcoA/eJ5OloIoqvCQi04X93kbmO62f/i2fqJgk62odrpl2ye04ue7tlKR93hckH4u46kVMoHQWmYF/rBWMA= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT /ipU68hnjrq1J7ZDi4JBaAyBiKjxZzpGZSVIfv8yFMKSKy3PpT/lba/0c4YCdqvrexVCCAL0MxBbob3A75l2Tnkgw/HHXtm0+abSHVywLEM= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT 1icyYJRr8366DjQYyqdj/TTYUgGOuUSp6ck6NDaYLdQ6lU3sbdOjY6CKYEN5T6XuJ8mBowWJicz/4l0lrv0Y+AZetgmFuW0OoM7yEd+5evQ= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT 2fa9YZ/g+IACqr1Etk+QIUPK51iiTwCA/t/nTw+U9RUmjfYorPLrzpPd46Bmj3C3tfw3cyOy5bx0ogGCOZ01eUctqHZQzXY903JlPvzPzlw= USPESNE STAZEN
ctiOS        - INFO     - POSIDENT 00ep7V5fFBZ

Podivame se na aktualizovanou databazi:

In [11]:
con = sqlite3.connect(temp_db)
cur = con.cursor()

for row in cur.execute("SELECT * FROM OPSUB order by ID LIMIT 10"):
    print(row)
    
con.close()

('+VKfYaKhJDrRHUWeS2e7xO7g+HhFQapZMPa5kamaF/wSOs6r3xEPEVFhtUlWqcXO9GTopZEjqpsTbsKhZmN7VxYJSxvgcgTWMGdxYkQVCaU=', 0, '2000-01-01T00:00:00', None, 3, 68574168010, None, None, None, None, 'OFO', 2, None, None, None, None, '470211573', None, 'Pavel', 'PAVEL', 'Lexa', 'LEXA', None, 6475, None, 'Úzká', 'Palkovice', 'Kopřivnice', None, None, 11000, None, 1, None, None, None, None, 105, '421391306')
('+lQ0Ne+Jhd5K4uvds1/KT8qqJcoA/eJ5OloIoqvCQi04X93kbmO62f/i2fqJgk62odrpl2ye04ue7tlKR93hckH4u46kVMoHQWmYF/rBWMA=', 0, '2000-01-01T00:00:00', None, 3, 52552937010, None, None, None, None, 'OFO', 2, None, None, None, None, '400504084', None, 'Milan', 'MILAN', 'Šíma', 'SIMA', None, 2578, None, 'Velká', 'Palkovice', 'Kopřivnice', None, None, 11000, None, 1, None, None, None, None, 64, '441241306')
('/AOrBGOFR+4GtqsAvOXC/3kKd0OCEPBVRfEsIPHeW3CoGcU1dio8gmrulsH5/KEvySRvbRT2hlDOXICKukDLldMgNY5z+yASe8F1rqBlKug=', 0, '2000-01-01T00:00:00', None, 3, 35088613010, None, None, None, None, 'OFO', 2, None, None, Non

Je dobre vedet, ze identifikatory nemusime ziskavat z databaze, ale muzeme je take ziskat z testoveho souboru, kde je oddelovacem carka:

In [13]:
txt_path = os.path.join(module_path, 'services', 'ctiOS', 'data', 'input', 'ctios_all.txt')
posidents = ctios.get_posidents_from_txt(txt_path)
print(posidents)

['<v2:pOSIdent>uecJ/wWk2Ej6CAnwe3i0y0jfrp9Xr1oMVJ+9kLKpkU8trb/GSsJmcvNw7XJ0dzNpkKLYrpaxDPIVMKGKnG/ZMzSYtEOoqKGnRHhbt/PXjUr/RJzL4O5LlsS30GNP3Kka</v2:pOSIdent>', '<v2:pOSIdent>\nVm+8vMmIsWsvcxR9zMDL9Cy+E0ZgHj9AJ3kwMqaLxguXn6wJcfM3e3LWpd7RzRxm3woikNUtGPF45crUFMpAsfsa/Bmx4YzTqTFCj0vhoQY=</v2:pOSIdent>', '<v2:pOSIdent>\nT/sOfNBmULfA5GzBpJRYu3UWEwhw4PFYydRQSh7x1PSc4RTRLXYaXTfpbt2Da17zSJPbQDKce+cHMMQVCVBRj9+U8wwRYYTBTgLsWbpMpuY=</v2:pOSIdent>', '<v2:pOSIdent>\nlWpFU3UmWlpSe8TtpUMRBCFflbRtrT1aEpzxnv1MEXnpCGhTicndBKbyJjsfMJMdgH+U78n9GU5DNm6bXxNhnC2JU2MIzwpJObcy4n2Baqk=</v2:pOSIdent>', '<v2:pOSIdent>\nmDkas+QvEOd4+XBxuIW4Da3aEEdBAod5OJM/9la0ivdkMk7DTs9DqRZEI5Lc3wAC7Yb1sw4kp/JEFP3nnG2ZcPT36EJYO+EfrC0OlzewTfM=</v2:pOSIdent>', '<v2:pOSIdent>\n7TsaZA7GFeEVxMscggvr2LkRAZwqfDywZ9HtkoX92fdNI/VFseD7SjfY8rJ7IlS7o2MG7D4mG56FZUTFOUWW7a5GeMmtDAasGLtodOgvIOA=</v2:pOSIdent>', '<v2:pOSIdent>\nyqpT3s/QygUWqCsg/azKypTOTOuJdbjB/16kQv0m5uugaHFJh8Mtp2i7sCdt71GRZixdYAzCVqEa4mNgvEjDECGf0Fby1B2nv1NdlyFQPrg=</v2:pOSIde

Nebo je muzeme rovnou inicializovat jako seznam, ktery predame jako argument do funkce ctios.process().